# Informal testing

This does a few tests of the code that are informal. I don't think most of them need to be codified into actual tests, since all we really care about is the final results.

In [1]:
import sys, os
sys.path.append(os.path.abspath("../"))

In [2]:
from art_enrich import lib as agb

Test that the function to find the index to interpolate in metallicity is correct.

In [3]:
agb.init_detailed_enrichment()

In [4]:
test_zs = [0, 0.000005, 0.0001, 0.0002, 0.00099, 0.001, 0.002, 
           0.005999, 0.006, 0.007, 0.009, 0.01, 0.015, 0.02, 0.03]

In [5]:
for z in test_zs:
    z_idxs = agb.find_z_bound_idxs_agb_py(z)
    print("{:8f} {} {}".format(z, z_idxs[0], z_idxs[1]))

0.000000 0 0
0.000005 0 0
0.000100 0 0
0.000200 0 1
0.000990 0 1
0.001000 1 1
0.002000 1 2
0.005999 1 2
0.006000 2 2
0.007000 2 3
0.009000 2 3
0.010000 3 3
0.015000 3 4
0.020000 4 4
0.030000 4 4


I'll do similar tests for age, both with and without the proper formula in `guess_age_idx`. This can't be done in line here in the notebook, since it has to run twice, but I'll just have to trust myself that I did it.

In [6]:
test_ages = [0, 29E6, 30E6, 40E6, 100E6, 1E9, 5E9, 10E9, 13.9E9, 14E9, 15E9, 1.32944e+08]

In [7]:
print("{:<12} {:<5} {:<5} {:<5} {:<12} {:<12}".format("age", "idx", "idx+", "idx_0", "age idx", "age idx+"))
for age in test_ages:
    age_idxs = agb.find_age_bound_idxs_agb_py(age)
    print("{:<12g} {:<5} {:<5} {:<5} {:<12g} {:<12g}".format(age, 
                                                             age_idxs[0], 
                                                             age_idxs[1], 
                                                             agb.guess_age_idx_agb_py(age),
                                                             agb.get_ages_agb(age_idxs[0]), 
                                                             agb.get_ages_agb(age_idxs[1])))

age          idx   idx+  idx_0 age idx      age idx+    
0            0     0     0     0            0           
2.9e+07      0     1     0     0            3e+07       
3e+07        1     1     1     3e+07        3e+07       
4e+07        47    48    47    3.99277e+07  4.01766e+07 
1e+08        194   195   194   9.95457e+07  1.00166e+08 
1e+09        565   566   565   9.98482e+08  1.00471e+09 
5e+09        824   825   824   4.99314e+09  5.02427e+09 
1e+10        935   936   935   9.95312e+09  1.00152e+10 
1.39e+10     988   989   988   1.38358e+10  1.39221e+10 
1.4e+10      989   990   989   1.39221e+10  1.40089e+10 
1.5e+10      1000  1000  999   1.49071e+10  1.49071e+10 
1.32944e+08  240   241   240   1.32488e+08  1.33313e+08 


This does appear to work. 

We can then see if the yields work! I'll do some simple checking here, with more detailed work in a separate file with unit tests.

In [8]:
for a in test_ages:
    for z in test_zs:
        rates = agb.get_ejecta_rate_agb_py(a, z)
        
        str_fmt = "age = {:<12g}, z={:<8f}\n".format(a, z)
        str_fmt += "{:<7g}  " * 5 + "\n\n"
        print(str_fmt.format(rates[0], rates[1], rates[2], rates[3], rates[4]))

age = 0           , z=0.000000
3.60668e-322  2.26583e-314  6.95321e-310  -1.70888e+292  2.26582e-314  


age = 0           , z=0.000005
3.60668e-322  2.26583e-314  6.95321e-310  -1.70888e+292  2.26582e-314  


age = 0           , z=0.000100
3.60668e-322  2.26583e-314  6.95321e-310  -1.70888e+292  2.26582e-314  


age = 0           , z=0.000200
3.60668e-322  2.26583e-314  6.95321e-310  -1.70888e+292  2.26582e-314  


age = 0           , z=0.000990
3.60668e-322  2.26583e-314  6.95321e-310  -1.70888e+292  2.26582e-314  


age = 0           , z=0.001000
3.60668e-322  2.26583e-314  6.95321e-310  -1.70888e+292  2.26582e-314  


age = 0           , z=0.002000
3.60668e-322  2.26583e-314  6.95321e-310  -1.70888e+292  2.26582e-314  


age = 0           , z=0.005999
3.60668e-322  2.26583e-314  6.95321e-310  -1.70888e+292  2.26582e-314  


age = 0           , z=0.006000
3.60668e-322  2.26583e-314  6.95321e-310  -1.70888e+292  2.26582e-314  


age = 0           , z=0.007000
3.60668e-322  2.26583e-3

In [9]:
rates[0]

6.95321042426857e-310

In [10]:
for idx in range(100):
    print(rates[idx])

2.2495326083e-314
2.2652962905e-314
6.9532104242678e-310
2.247655101e-314
5e-324
2.248894801e-314
2.2495326083e-314
2.2488120886e-314
2.2488697975e-314
2.2488698014e-314
6.9532104242725e-310
2.2475198935e-314
0.0
2.265300476e-314
2.250268944e-314
2.2488697975e-314
2.2652586427e-314
2.57e-322
2.2652586427e-314
1.24e-322
6.9532104242749e-310
2.2663423984e-314
2.2488840937e-314
2.2652586427e-314
2.249030622e-314
1.24e-322
-1.5135017178310236e+245
5e-322
2.249030685e-314
1.24e-322
6.953210424301e-310
2.248175337e-314
0.0
2.2658717268e-314
8e-323
1.315637390744e-312
1.35807730653e-312
1.40051722236e-312
5.51718905784e-313
4.24399158287e-313
3.182993687e-313
2.1219957943e-313
1.59149684334e-312
1.67637667525e-312
5.0927898994e-313
1.612716801257e-312
2.121995829e-314
1.84613633858e-312
1.549056927686e-312
1.251977517015e-312
1.167097685387e-312
6.15378779657e-313
5.9415882182e-313
1.27319747488e-312
2.26583117e-314
2.121995792e-314
1.87283945149e-312
2e-323
2.2657402855e-314
2.26588511e-314
